When using Word2Vec, you're making the assumption of the distributional hypothesis: **the meaning of a word can be inferred by the company it keeps**. 

In this sense, if you have two words with very similar neighbors, then these words are probably similar in meaning. 

In this tutorial, we will use the Gensim implementation of Word2Vec and we will get it working. 

For this tutorial, we will use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/#.XUkA6nVKhhE) dataset. This dataset has car and hotel reviews. We will compress the hotel reviews into a single file and use them for our model. 

In [3]:
import gzip
import logging
import gensim

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s",\
                   level=logging.INFO)